<a href="https://colab.research.google.com/github/Dimasputraangrh/bunga/blob/main/ANN_PREDIKSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

TAMPILKAN DATASET

In [15]:
# Load dataset
data = pd.read_csv("/content/youtube-top-100-songs-2025 (1).csv")

# Lihat 5 data pertama
display(data.head())

# Cek struktur data
data.info()


,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...,2009014557,Music,YG Entertainment;YG;와이지;K-pop;BLACKPINK;블랙핑크;블...,173,2:53,False,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,ROSÉ,https://www.youtube.com/channel/UCBo1hnzxV9rz3...,19200000
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...","Lady Gaga, Bruno Mars - Die With A Smile (Offi...",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...,1324833300,Music,Lady Gaga;Bruno Mars;Interscope;Pop,252,4:12,False,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
2,Reneé Rapp - Leave Me Alone (Official Music Vi...,Reneé Rapp - Leave Me Alone (Official Music Vi...,"Listen to “BITE ME”, the new album from Reneé ...",2536628,Music,Reneé Rapp;Interscope Records;Pop,160,2:40,False,https://i.ytimg.com/vi/tiPWzFLiz4A/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,Billie Eilish - BIRDS OF A FEATHER (Official M...,Listen to HIT ME HARD AND SOFT: https://billie...,558329099,Music,Billie Eilish;Darkroom/Interscope Records;Alte...,231,3:51,False,https://i.ytimg.com/vi/V9PVRfjEBTI/maxresdefau...,Billie Eilish,https://www.youtube.com/channel/UCDGmojLIoWpXo...,56800000
4,Reneé Rapp - Mad (Official Music Video),Reneé Rapp - Mad (Official Music Video),"Listen to “BITE ME”, the new album from Reneé ...",2113548,Music,Reneé Rapp;Interscope Records;Pop,180,3:00,False,https://i.ytimg.com/vi/xkWQM3flsiY/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   100 non-null    object
 1   fulltitle               100 non-null    object
 2   description             100 non-null    object
 3   view_count              100 non-null    int64 
 4   categories              100 non-null    object
 5   tags                    85 non-null     object
 6   duration                100 non-null    int64 
 7   duration_string         100 non-null    object
 8   live_status             100 non-null    bool  
 9   thumbnail               100 non-null    object
 10  channel                 100 non-null    object
 11  channel_url             100 non-null    object
 12  channel_follower_count  100 non-null    int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 9.6+ KB


LOAD

In [16]:
df = pd.read_csv("/content/youtube-top-100-songs-2025 (1).csv")

# ===== KATEGORI DURASI =====
def kategori_durasi(d):
    if d < 180:
        return "Pendek"
    elif d <= 300:
        return "Sedang"
    else:
        return "Panjang"

df['durasi_kategori'] = df['duration'].apply(kategori_durasi)

# ===== KELAS VIEW =====
conditions = [
    df['view_count'] < 100_000_000,
    (df['view_count'] >= 100_000_000) & (df['view_count'] < 1_000_000_000),
    df['view_count'] >= 1_000_000_000
]

choices = ['Rendah', 'Sedang', 'Tinggi']
df['kelas_view'] = np.select(conditions, choices, default='Lainnya')

INPUT DAN ENCODING

In [8]:
X = df[['categories', 'durasi_kategori', 'channel']]
y = df['kelas_view']

# Encode label output
label_y = LabelEncoder()
y_enc = label_y.fit_transform(y)
y_cat = to_categorical(y_enc)

# OneHot + Label Encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'),
         ['categories', 'durasi_kategori', 'channel'])
    ]
)

X_processed = preprocessor.fit_transform(X)

# Normalisasi
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_processed)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_cat, test_size=0.2, random_state=42
)

MODEL ANN

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         4,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,723 (26.26 KB)

 Trainable params: 6,723 (26.26 KB)

 Non-trainable params: 0 (0.00 B)

TRAINING

In [10]:
model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=8,
    validation_split=0.1,
    verbose=1
)

Epoch 1/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.1094 - loss: 1.7486 - val_accuracy: 0.5000 - val_loss: 0.8426
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5690 - loss: 0.9133 - val_accuracy: 0.7500 - val_loss: 0.6281
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7413 - loss: 0.6300 - val_accuracy: 0.7500 - val_loss: 0.5884
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6896 - loss: 0.6561 - val_accuracy: 0.7500 - val_loss: 0.5740
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7250 - loss: 0.5555 - val_accuracy: 0.7500 - val_loss: 0.5560
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7577 - loss: 0.5030 - val_accuracy: 0.7500 - val_loss: 0.5522
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7642 - loss: 0.4308 - val_accuracy: 0.7500 - val_loss: 0.5480
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8733 - loss: 0.3244 - val_accuracy: 0.7500 - val_loss:

INPUT MANUAL

In [17]:
print("=== INPUT MANUAL ===")
kategori = input("Masukkan kategori musik (contoh: Music): ")
durasi_input = input("Masukkan durasi (Pendek/Sedang/Panjang): ")
penyanyi = input("Masukkan nama penyanyi (harus ada di data): ")

# Normalize input to match the casing of training data
kategori_normalized = kategori.title()
durasi_input_normalized = durasi_input.title()
# Attempt to normalize channel name as well, assuming a common casing pattern
# Note: For 'channel', it's crucial that the exact string (including casing) exists in the training data.
penyanyi_normalized = penyanyi.title()

input_df = pd.DataFrame({
    'categories': [kategori_normalized],
    'durasi_kategori': [durasi_input_normalized],
    'channel': [penyanyi_normalized]
})

# Transform input
input_proc = preprocessor.transform(input_df)
input_scaled = scaler.transform(input_proc)

# Prediksi
pred = model.predict(input_scaled)
kelas = label_y.inverse_transform([np.argmax(pred)])

print("\nHasil Prediksi View:", kelas[0])
print("Probabilitas:", pred)

=== INPUT MANUAL ===
Masukkan kategori musik (contoh: Music): MUSIC
Masukkan durasi (Pendek/Sedang/Panjang): PENDEK
Masukkan nama penyanyi (harus ada di data): Reneé Rapp	
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

Hasil Prediksi View: Rendah
Probabilitas: [[0.9273532  0.06680043 0.00584635]]
